<table><tr><td>Run the latest release of this notebook:</td><td><a href="https://mybinder.org/v2/gh/stellargraph/stellargraph/master?urlpath=lab/tree/demos/connector/neo4j/cluster-gcn-on-cora-neo4j-example.ipynb" alt="Open In Binder" target="_parent"><img src="https://mybinder.org/badge_logo.svg"/></a></td><td><a href="https://colab.research.google.com/github/stellargraph/stellargraph/blob/master/demos/connector/neo4j/cluster-gcn-on-cora-neo4j-example.ipynb" alt="Open In Colab" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg"/></a></td></tr></table>

In [1]:
# install StellarGraph if running on Google Colab
import sys
if 'google.colab' in sys.modules:
  %pip install -q stellargraph[demos]==1.1.0b

In [2]:
# verify that we're using the correct version of StellarGraph for this notebook
import stellargraph as sg

try:
    sg.utils.validate_notebook_version("1.1.0b")
except AttributeError:
    raise ValueError(
        f"This notebook requires StellarGraph version 1.1.0b, but a different version {sg.__version__} is installed.  Please see <https://github.com/stellargraph/stellargraph/issues/1172>."
    ) from None

In [3]:
import stellargraph as sg
from stellargraph.connector.neo4j import Neo4jClusterNodeGenerator, Neo4jStellarGraph
from stellargraph.layer import ClusterGCN
import tensorflow as tf
import py2neo
import os
from sklearn import preprocessing, feature_extraction, model_selection

import numpy as np
import scipy.sparse as sps
import pandas as pd

In [4]:
default_host = os.environ.get("STELLARGRAPH_NEO4J_HOST")

# Create the Neo4j Graph database object;
# the arguments can be edited to specify location and authentication
graph = py2neo.Graph(host=default_host, port=None, user=None, password="pass")
neo4j_sg = Neo4jStellarGraph(graph)

In [5]:
# read the node labels from a seperate file
# note this function also returns a StellarGraph
# which we won't be using for this demo - we only need Neo4jStellarGraph!
_, labels = sg.datasets.Cora().load()

In [6]:
# split the node labels into train/test/val

train_labels, test_labels = model_selection.train_test_split(
    labels, train_size=140, test_size=None, stratify=labels
)
val_labels, test_labels = model_selection.train_test_split(
    test_labels, train_size=500, test_size=None, stratify=test_labels
)

target_encoding = preprocessing.LabelBinarizer()

train_targets = target_encoding.fit_transform(train_labels)
val_targets = target_encoding.transform(val_labels)
test_targets = target_encoding.transform(test_labels)

In [7]:
# create the Neo4jClusterNodeGenerator
# and the keras sequence objects

generator = Neo4jClusterNodeGenerator(neo4j_sg, clusters=5)

train_gen = generator.flow(train_labels.index, targets=train_targets)
val_gen = generator.flow(val_labels.index, targets=val_targets)
test_gen = generator.flow(test_labels.index, targets=test_targets)

Number of clusters 5
0 cluster has size 541
1 cluster has size 541
2 cluster has size 541
3 cluster has size 541
4 cluster has size 544


In [8]:
# create the model

cluster_gcn = ClusterGCN(
    layer_sizes=[32, 32], generator=generator, activations=["relu", "relu"], dropout=0.5,
)

x_in, x_out = cluster_gcn.in_out_tensors()
predictions = tf.keras.layers.Dense(units=val_targets.shape[1], activation="softmax")(
    x_out
)
model = tf.keras.Model(x_in, predictions)
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.01),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=["acc"],
)

In [9]:
# train the model!
model.fit(train_gen, validation_data=val_gen, epochs=10)

  ['...']
  ['...']
Train for 5 steps, validate for 5 steps
Epoch 1/10
5/5 [==============================] - 10s 2s/step - loss: 1.9248 - acc: 0.2571 - val_loss: 1.8326 - val_acc: 0.3000
Epoch 2/10
5/5 [==============================] - 10s 2s/step - loss: 1.6974 - acc: 0.3214 - val_loss: 1.7369 - val_acc: 0.3060
Epoch 3/10
5/5 [==============================] - 10s 2s/step - loss: 1.5327 - acc: 0.4143 - val_loss: 1.5998 - val_acc: 0.4140
Epoch 4/10
5/5 [==============================] - 10s 2s/step - loss: 1.3429 - acc: 0.5214 - val_loss: 1.4707 - val_acc: 0.4980
Epoch 5/10
5/5 [==============================] - 10s 2s/step - loss: 1.0642 - acc: 0.6643 - val_loss: 1.3391 - val_acc: 0.5340
Epoch 6/10
5/5 [==============================] - 10s 2s/step - loss: 0.7864 - acc: 0.7643 - val_loss: 1.2693 - val_acc: 0.5800
Epoch 7/10
5/5 [==============================] - 11s 2s/step - loss: 0.5935 - acc: 0.7929 - val_loss: 1.2978 - val_acc: 0.5720
Epoch 8/10
5/5 [============================

In [10]:
# evaluate the model
model.evaluate(test_gen)

  ['...']
5/5 [==============================] - 5s 980ms/step - loss: 1.2825 - acc: 0.6272


[1.2825486660003662, 0.627176]

<table><tr><td>Run the latest release of this notebook:</td><td><a href="https://mybinder.org/v2/gh/stellargraph/stellargraph/master?urlpath=lab/tree/demos/connector/neo4j/cluster-gcn-on-cora-neo4j-example.ipynb" alt="Open In Binder" target="_parent"><img src="https://mybinder.org/badge_logo.svg"/></a></td><td><a href="https://colab.research.google.com/github/stellargraph/stellargraph/blob/master/demos/connector/neo4j/cluster-gcn-on-cora-neo4j-example.ipynb" alt="Open In Colab" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg"/></a></td></tr></table>